In [ ]:
import pandas as pd
import numpy as np
import json
from scipy import stats
from matplotlib import pyplot
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir = ''

In [ ]:
with open(f'{dir}/type_ids.json', 'r') as f:
    type_ids = json.load(f)
task_sample_ratio = 0.3
input_embedding_mode = 'fasttext'

input_dir = f'{dir}/tag_set_extension/{len(type_ids)}_way/{task_sample_ratio}/{input_embedding_mode}'

In [ ]:
letters = ["a", "b", "c"]

# Data split
for letter in letters:
    # Iteration
    for i in range(3):
        result = pd.read_csv(f"{input_dir}/train_log_{letter}_iter_{i}.csv")
        f1 = result["meta_model_validation_f1_score_percent"].iloc[-1]
        f1_base = result["meta_model_validation_f1_score_percent_base"].iloc[-1]

# Collect data
table_rows = []
for letter in letters:
    f1_scores = []
    f1_base_scores = []
    f1_scores_full = []
    f1_base_scores_full = []

    for i in range(3):
        result = pd.read_csv(f"{input_dir}/train_log_{letter}_iter_{i}.csv")
        result_full = pd.read_csv(f"{input_dir}/train_log_{letter}_full_iter_{i}.csv")
        f1_scores.append(result["meta_model_validation_f1_score_percent"].iloc[-1])
        f1_base_scores.append(result["meta_model_validation_f1_score_percent_base"].iloc[-1])
        f1_scores_full.append(result_full["meta_model_validation_f1_score_percent"].iloc[-1])
        f1_base_scores_full.append(result_full["meta_model_validation_f1_score_percent_base"].iloc[-1])

    # Compute mean and 95% confidence interval
    mean_f1 = np.mean(f1_scores)
    mean_f1_base = np.mean(f1_base_scores)
    ci_f1 = stats.t.interval(0.95, len(f1_scores)-1, loc=mean_f1, scale=stats.sem(f1_scores))
    ci_f1_base = stats.t.interval(0.95, len(f1_base_scores)-1, loc=mean_f1_base, scale=stats.sem(f1_base_scores))

    mean_f1_full = np.mean(f1_scores_full)
    mean_f1_base_full = np.mean(f1_base_scores_full)
    ci_f1_full = stats.t.interval(0.95, len(f1_scores_full)-1, loc=mean_f1_full, scale=stats.sem(f1_scores_full))
    ci_f1_base_full = stats.t.interval(0.95, len(f1_base_scores_full)-1, loc=mean_f1_base_full, scale=stats.sem(f1_base_scores_full))

    # Add normal row
    row_normal = f"{letter} & No & {mean_f1:.2f} [{ci_f1[0]:.2f}, {ci_f1[1]:.2f}] & {mean_f1_base:.2f} [{ci_f1_base[0]:.2f}, {ci_f1_base[1]:.2f}] \\\\"
    # Add full row
    row_full = f"{letter} & Yes & {mean_f1_full:.2f} [{ci_f1_full[0]:.2f}, {ci_f1_full[1]:.2f}] & {mean_f1_base_full:.2f} [{ci_f1_base_full[0]:.2f}, {ci_f1_base_full[1]:.2f}] \\\\"

    table_rows.extend([row_normal, row_full])

# Print full LaTeX table
latex_table = r"""\begin{table}[h!]
\centering
\caption{Meta-model validation F1 scores across 3 iterations (mean $\pm$ 95\% CI), including full dataset scores.}
\label{tab:f1_summary}
\begin{tabular}{c c c c}
\toprule
Letter & Full? & F1 Score (\%) & Base F1 (\%) \\
\midrule
""" + "\n".join(table_rows) + r"""
\bottomrule
\end{tabular}
\end{table}"""

print(latex_table)

\begin{table}[h!]
\centering
\caption{Meta-model validation F1 scores across 3 iterations (mean $\pm$ 95\% CI), including full dataset scores.}
\label{tab:f1_summary}
\begin{tabular}{c c c c}
\toprule
Letter & Full? & F1 Score (\%) & Base F1 (\%) \\
\midrule
a & No & 51.36 [47.74, 54.98] & 51.36 [47.74, 54.98] \\
a & Yes & 52.47 [49.25, 55.69] & 56.21 [53.36, 59.06] \\
b & No & 55.53 [51.55, 59.51] & 55.53 [51.55, 59.51] \\
b & Yes & 52.43 [52.27, 52.60] & 54.30 [53.48, 55.13] \\
c & No & 52.34 [44.31, 60.37] & 53.59 [50.87, 56.31] \\
c & Yes & 51.06 [48.10, 54.02] & 51.70 [50.55, 52.86] \\
\bottomrule
\end{tabular}
\end{table}


In [ ]:
input_dir = f'{dir}/tag_set_extension/{len(type_ids)}_way/{task_sample_ratio}'
for letter in ["a", "b", "c"]:
    with open(f'{input_dir}/{letter}_task_set_type_id_list_of_samples.json', 'r') as f:
        task_set_type_id_list_of_samples = json.load(f)

    print(f"split {letter} type_ids:")
    for type_id, list_of_samples in task_set_type_id_list_of_samples.items():
        print(type_id, type_ids[int(type_id)])

split a type_ids:
0 Activity
2 Behavior
3 Conceptual Entity
4 Finding
5 Group
9 Manufactured Object
10 Natural Phenomenon or Process
12 Occupational Activity
13 Organism
14 Organism Attribute
16 Phenomenon or Process
17 Substance
18 UnknownType
split b type_ids:
0 Activity
1 Anatomical Structure
2 Behavior
4 Finding
5 Group
6 Idea or Concept
7 Injury or Poisoning
8 Intellectual Product
11 Occupation or Discipline
12 Occupational Activity
13 Organism
15 Organization
18 UnknownType
split c type_ids:
1 Anatomical Structure
3 Conceptual Entity
6 Idea or Concept
7 Injury or Poisoning
8 Intellectual Product
9 Manufactured Object
10 Natural Phenomenon or Process
11 Occupation or Discipline
14 Organism Attribute
15 Organization
16 Phenomenon or Process
17 Substance
18 UnknownType


In [ ]:
input_dir = f'{dir}/tag_set_extension/{len(type_ids)}_way/{task_sample_ratio}'
all_type_ids = set(list(range(19)))

for letter in ["a", "b", "c"]:
    with open(f'{input_dir}/{letter}_task_set_type_id_list_of_samples.json', 'r') as f:
        task_set_type_id_list_of_samples = json.load(f)

    present_ids = set(map(int, task_set_type_id_list_of_samples.keys()))
    missing_ids = all_type_ids - present_ids

    print(f"split {letter} missing type_ids:")
    for type_id in sorted(missing_ids):
        print(type_id, type_ids[type_id])

split a missing type_ids:
1 Anatomical Structure
6 Idea or Concept
7 Injury or Poisoning
8 Intellectual Product
11 Occupation or Discipline
15 Organization
split b missing type_ids:
3 Conceptual Entity
9 Manufactured Object
10 Natural Phenomenon or Process
14 Organism Attribute
16 Phenomenon or Process
17 Substance
split c missing type_ids:
0 Activity
2 Behavior
4 Finding
5 Group
12 Occupational Activity
13 Organism


In [ ]:
from google.colab import runtime

runtime.unassign()